In [76]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import random

In [77]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [78]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [79]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

56 documents
12 classes ['entertainment_movies', 'entertainment_music', 'flight_assistant', 'goodbye', 'greeting', 'options', 'order_dessert', 'order_drinks', 'order_food', 'tellmejoke', 'thanks', 'weather_search']
86 unique lemmatized words ["'s", ',', 'a', 'album', 'all', 'am', 'anyone', 'are', 'assistant', 'available', 'awesome', 'be', 'beverage', 'bored', 'bye', 'call', 'can', 'chatting', 'could', 'day', 'dessert', 'do', 'doubt', 'drink', 'eat', 'flight', 'food', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hi', 'how', 'i', 'in', 'is', 'item', 'joke', 'later', 'like', 'list', 'listen', 'me', 'menu', 'movie', 'music', 'navigate', 'need', 'next', 'nice', 'of', 'offered', 'on', 'option', 'order', 'outside', 'play', 'provide', 'see', 'show', 'some', 'song', 'support', 'talk', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'today', 'want', 'watch', 'weather', 'what', 'where', 'which', 'you']


In [80]:

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


In [81]:

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [82]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [83]:

# save all of our data structures
print("Saving training data")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data.parquet", "wb" ) )

Saving training data
